# EDA For Mapping Vulnerabilities

Lets import our dependencies

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium as flm
from sklearn.decomposition import PCA

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

And read in the dataset.

In [2]:
df = pd.read_csv('main.csv', index_col=None)
df

,psgc,name,city_munic,province,clean_idx,longitude,latitude,coords,geometry,FID,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,cost_of_doing_business,cost_of_living,disaster_risk_reduction_plan,distance_to_ports,early_warning_system,education,emergency_infrastructure,employed_population,employment_generation,financial_deepening,financial_technology_capacity,getting_business_permits,health,information_technology_capacity,lgu_investment,land_use_plan,local_economy_growth,local_economy_size,local_risk_assessments,peace_and_order,presence_of_business_and_professional_organizations,presence_of_investment_promotion_unit,productivity,recognition_of_performance,road_network,safety_compliant_business,sanitary_system,social_protection,transportation_vehicles,utilities
0,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2017.0,0.0000,0.2120,2.5000,2.5000,0.0017,0.1098,0.2879,1.4320,2.2597,2.3718,2.2277,1.5203,2.5000,2.1655,2.5000,0.0997,0.8750,0.0457,0.0218,0.1894,0.0463,0.7434,0.0623,0.2976,0.4592,1.8750,0.0012,0.0039,2.5000,0.4161,0.0000,2.5000,0.0145,0.2879,0.0000,0.1943,1.5706,0.1104,0.0204,0.9949
1,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2018.0,0.0000,0.2337,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.1771,1.9907,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0197,0.0423,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0224,0.0072,0.0000,0.0000,0.0218,0.0000,0.0047,0.0000,0.0000,0.1253,0.0000,0.0000,0.0000,0.0000
2,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2019.0,0.0000,0.0000,2.2500,2.1047,0.2189,0.0614,0.1451,0.0074,1.9499,2.2000,1.9655,0.9127,2.4537,2.3204,2.5000,0.1300,0.4984,0.0401,0.0000,0.1339,0.0671,2.3451,0.0689,0.0806,0.3137,2.4231,0.0024,0.0051,2.5000,0.0012,0.0007,2.5000,0.0004,0.0595,0.1006,0.2553,1.4921,0.0007,0.0029,1.8558
3,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2020.0,0.0000,0.1653,2.5000,2.3125,0.0852,0.0587,0.0488,0.0083,2.2404,2.4242,2.2798,0.3017,2.5000,2.4933,2.5000,0.1185,0.5004,0.0385,0.0651,0.1390,0.0631,2.3385,0.0830,0.2679,0.1665,2.5000,0.1419,0.0013,2.5000,0.0072,0.0216,0.6250,0.0413,0.0023,0.0006,0.1643,0.9946,0.0024,0.0026,1.8706
4,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2021.0,0.0000,0.2606,1.2542,2.2948,0.0215,0.1064,0.0145,0.0061,2.1365,2.2619,2.3654,1.6250,2.4737,2.3721,1.2537,0.0835,0.0531,0.0060,0.1052,0.1231,0.0436,1.8169,0.0816,0.0909,0.0220,2.4545,0.0045,0.0067,2.5000,0.0283,0.1089,1.8750,0.0690,0.0000,0.1846,0.1683,1.2502,0.1047,0.0030,1.2478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,806023000,Zumarraga,Zumarraga,Samar,"zumarraga, samar",124.841531,11.639055,"11.6390554, 124.8415308",MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,Zumarraga,2018.0,0.0000,0.0004,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.8120,1.0185,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9800,806023000,Zumarraga,Zumarraga,Samar,"zumarrag

In [3]:
df.shape

(9804, 52)

In [4]:
type(df)

pandas.core.frame.DataFrame

In [5]:
# df = gpd.GeoDataFrame(df, geometry=df['geometry'])
# type(df)

Lets check for **data types** and **Null** values.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9804 entries, 0 to 9803
Data columns (total 52 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   psgc                                                 9804 non-null   int64  
 1   name                                                 9804 non-null   object 
 2   city_munic                                           9804 non-null   object 
 3   province                                             9804 non-null   object 
 4   clean_idx                                            9804 non-null   object 
 5   longitude                                            9804 non-null   float64
 6   latitude                                             9804 non-null   float64
 7   coords                                               9804 non-null   object 
 8   geometry                                             9756 non-null  

We have a number of missing values and data type discrepancies.  
  
Lets start with checking the missing values.

In [7]:
null_rows = df[df['geometry'].isnull()]
null_rows['name'].unique()

array(['Datu Hoffer Ampatuan', 'Datu Salibo', 'Hadji Muhtamad', 'Igacos',
       'Kalayaan (PN)', 'Omar', 'Shariff Saydona Mustapha', 'Tabuan-Lasa'],
      dtype=object)

We have 8 cities missing geometry files.  
  
Lets fill these with 'Not Available' to indicate it has missing geometry.

In [8]:
df['geometry'].fillna('Not Available', inplace=True)

A quick check we have resolved this.

In [9]:
null_rows = df[df['geometry'].isnull()]
null_rows

,psgc,name,city_munic,province,clean_idx,longitude,latitude,coords,geometry,FID,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,cost_of_doing_business,cost_of_living,disaster_risk_reduction_plan,distance_to_ports,early_warning_system,education,emergency_infrastructure,employed_population,employment_generation,financial_deepening,financial_technology_capacity,getting_business_permits,health,information_technology_capacity,lgu_investment,land_use_plan,local_economy_growth,local_economy_size,local_risk_assessments,peace_and_order,presence_of_business_and_professional_organizations,presence_of_investment_promotion_unit,productivity,recognition_of_performance,road_network,safety_compliant_business,sanitary_system,social_protection,transportation_vehicles,utilities


In [10]:
null_rows2 = df[df['city_municipality'].isnull()]
null_rows2['name'].unique()

array(["Brooke's Point", "T'Boli"], dtype=object)

We have two cities with missing information.  
  
Let's use **'name'** to fill the missing **'city_municipality'** data.

In [11]:
df['city_municipality'].fillna(df['name'], inplace=True)

In [12]:
null_rows = df[df['city_municipality'].isnull()]
null_rows

,psgc,name,city_munic,province,clean_idx,longitude,latitude,coords,geometry,FID,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,cost_of_doing_business,cost_of_living,disaster_risk_reduction_plan,distance_to_ports,early_warning_system,education,emergency_infrastructure,employed_population,employment_generation,financial_deepening,financial_technology_capacity,getting_business_permits,health,information_technology_capacity,lgu_investment,land_use_plan,local_economy_growth,local_economy_size,local_risk_assessments,peace_and_order,presence_of_business_and_professional_organizations,presence_of_investment_promotion_unit,productivity,recognition_of_performance,road_network,safety_compliant_business,sanitary_system,social_protection,transportation_vehicles,utilities


In [13]:
rows = df[df['name'] == "Brooke's Point"]
rows

,psgc,name,city_munic,province,clean_idx,longitude,latitude,coords,geometry,FID,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,cost_of_doing_business,cost_of_living,disaster_risk_reduction_plan,distance_to_ports,early_warning_system,education,emergency_infrastructure,employed_population,employment_generation,financial_deepening,financial_technology_capacity,getting_business_permits,health,information_technology_capacity,lgu_investment,land_use_plan,local_economy_growth,local_economy_size,local_risk_assessments,peace_and_order,presence_of_business_and_professional_organizations,presence_of_investment_promotion_unit,productivity,recognition_of_performance,road_network,safety_compliant_business,sanitary_system,social_protection,transportation_vehicles,utilities
1458,1705306000,Brooke's Point,Brooke'S Point,Palawan,"brookespoint, palawan",117.838979,8.773602,"8.7736019, 117.8389787",POLYGON ((117.81459045400015 9.019559860000186...,243,Brooke's Point,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1459,1705306000,Brooke's Point,Brooke'S Point,Palawan,"brookespoint, palawan",117.838979,8.773602,"8.7736019, 117.8389787",POLYGON ((117.81459045400015 9.019559860000186...,243,Brooke's Point,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,1705306000,Brooke's Point,Brooke'S Point,Palawan,"brookespoint, palawan",117.838979,8.773602,"8.7736019, 117.8389787",POLYGON ((117.81459045400015 9.019559860000186...,243,Brooke's Point,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,1705306000,Brooke's Point,Brooke'S Point,Palawan,"brookespoint, palawan",117.838979,8.773602,"8.7736019, 117.8389787",POLYGON ((117.81459045400015 9.019559860000186...,243,Brooke's Point,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,1705306000,Brooke's Point,Brooke'S Point,Palawan,"brookespoint, palawan",117.838979,8.773602,"8.7736019, 117.8389787",POLYGON ((117.81459045400015 9.019559860000186...,243,Brooke's Point,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1463,1705306000,Brooke's Point,Brooke'S Point,Palawan,"brookespoint, palawan",117.838979,8.773602,"8.7736019, 117.8389787",POLYGON ((117.81459045400015 9.019559860000186...,243,Brooke's Point,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
rows = df[df['name'] == "T'Boli"]
rows

,psgc,name,city_munic,province,clean_idx,longitude,latitude,coords,geometry,FID,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,cost_of_doing_business,cost_of_living,disaster_risk_reduction_plan,distance_to_ports,early_warning_system,education,emergency_infrastructure,employed_population,employment_generation,financial_deepening,financial_technology_capacity,getting_business_permits,health,information_technology_capacity,lgu_investment,land_use_plan,local_economy_growth,local_economy_size,local_risk_assessments,peace_and_order,presence_of_business_and_professional_organizations,presence_of_investment_promotion_unit,productivity,recognition_of_performance,road_network,safety_compliant_business,sanitary_system,social_protection,transportation_vehicles,utilities
9252,1206316000,T'Boli,T'Boli,South Cotabato,"tboli, southcotabato",124.822925,6.213832,"6.2138317, 124.8229251",POLYGON ((124.79447937000009 6.241089821000116...,1542,T'Boli,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9253,1206316000,T'Boli,T'Boli,South Cotabato,"tboli, southcotabato",124.822925,6.213832,"6.2138317, 124.8229251",POLYGON ((124.79447937000009 6.241089821000116...,1542,T'Boli,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9254,1206316000,T'Boli,T'Boli,South Cotabato,"tboli, southcotabato",124.822925,6.213832,"6.2138317, 124.8229251",POLYGON ((124.79447937000009 6.241089821000116...,1542,T'Boli,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9255,1206316000,T'Boli,T'Boli,South Cotabato,"tboli, southcotabato",124.822925,6.213832,"6.2138317, 124.8229251",POLYGON ((124.79447937000009 6.241089821000116...,1542,T'Boli,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9256,1206316000,T'Boli,T'Boli,South Cotabato,"tboli, southcotabato",124.822925,6.213832,"6.2138317, 124.8229251",POLYGON ((124.79447937000009 6.241089821000116...,1542,T'Boli,2021.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9257,1206316000,T'Boli,T'Boli,South Cotabato,"tboli, southcotabato",124.822925,6.213832,"6.2138317, 124.8229251",POLYGON ((124.79447937000009 6.241089821000116...,1542,T'Boli,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df['city_municipality'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 9804 entries, 0 to 9803
Series name: city_municipality
Non-Null Count  Dtype 
--------------  ----- 
9804 non-null   object
dtypes: object(1)
memory usage: 76.7+ KB


We know we have lots of zeros, so lets count them. First we need to change the columns from **object** to **float**.  
  
The conversion to **floats** failed due to there being **'-'** in the columns.  
  
Lets see where they are and how many.

In [16]:
dash_by_column = (df == '-').sum() / 100
dash_by_column

psgc                                                   0.00
name                                                   0.00
city_munic                                             0.00
province                                               0.00
clean_idx                                              0.00
longitude                                              0.00
latitude                                               0.00
coords                                                 0.00
geometry                                               0.00
FID                                                    0.00
city_municipality                                      0.00
year                                                   0.00
accommodation_capacity                                 5.40
active_establishments_in_the_locality                  5.51
annual_disaster_drill                                  5.40
availability_of_basic_utilities                        5.40
budget_for_drrmp                        

We can see that they are all in the DTi features.  
  
Lets convert the '-' to 0.000 to match the other missing values.

In [17]:
df.replace('-', 0.0000, inplace=True)

Lets convert the DTi columns to `float`.

In [18]:
df[df.columns[12:51]] = df[df.columns[12:51]].astype(float)

And the years to `int`.

In [19]:
df['year'] = df['year'].astype(int)

Lets have a count of the zero's.

In [20]:
zeros_by_column = df.eq(0).sum()
zeros_by_column

psgc                                                      0
name                                                      0
city_munic                                                0
province                                                  0
clean_idx                                                 0
longitude                                                 0
latitude                                                  0
coords                                                    0
geometry                                                  0
FID                                                       6
city_municipality                                         0
year                                                      0
accommodation_capacity                                 5621
active_establishments_in_the_locality                  1481
annual_disaster_drill                                  2628
availability_of_basic_utilities                        2203
budget_for_drrmp                        

Lets convert all the zero's to **Null Values**.

In [21]:
df.loc[:,df.columns[12:52]] = df.loc[:,df.columns[12:52]].replace(0, np.nan)

And check the result.

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9804 entries, 0 to 9803
Data columns (total 52 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   psgc                                                 9804 non-null   int64  
 1   name                                                 9804 non-null   object 
 2   city_munic                                           9804 non-null   object 
 3   province                                             9804 non-null   object 
 4   clean_idx                                            9804 non-null   object 
 5   longitude                                            9804 non-null   float64
 6   latitude                                             9804 non-null   float64
 7   coords                                               9804 non-null   object 
 8   geometry                                             9804 non-null  

In [23]:
df2 = df.fillna(method='ffill').fillna(method='bfill')

In [24]:
df2

,psgc,name,city_munic,province,clean_idx,longitude,latitude,coords,geometry,FID,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,cost_of_doing_business,cost_of_living,disaster_risk_reduction_plan,distance_to_ports,early_warning_system,education,emergency_infrastructure,employed_population,employment_generation,financial_deepening,financial_technology_capacity,getting_business_permits,health,information_technology_capacity,lgu_investment,land_use_plan,local_economy_growth,local_economy_size,local_risk_assessments,peace_and_order,presence_of_business_and_professional_organizations,presence_of_investment_promotion_unit,productivity,recognition_of_performance,road_network,safety_compliant_business,sanitary_system,social_protection,transportation_vehicles,utilities
0,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2017,0.0072,0.2120,2.5000,2.5000,0.0017,0.1098,0.2879,1.4320,2.2597,2.3718,2.2277,1.5203,2.5000,2.1655,2.5000,0.0997,0.8750,0.0457,0.0218,0.1894,0.0463,0.7434,0.0623,0.2976,0.4592,1.8750,0.0012,0.0039,2.5,0.4161,0.0218,2.5000,0.0145,0.2879,0.1006,0.1943,1.5706,0.1104,0.0204,0.9949
1,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2018,0.0072,0.2337,2.5000,2.5000,0.0017,0.1098,0.2879,1.4320,2.2597,2.3718,2.1771,1.9907,2.5000,2.1655,2.5000,0.0997,0.8750,0.0457,0.0197,0.0423,0.0463,0.7434,0.0623,0.2976,0.4592,1.8750,0.0224,0.0072,2.5,0.4161,0.0218,2.5000,0.0047,0.2879,0.1006,0.1253,1.5706,0.1104,0.0204,0.0000
2,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2019,0.0072,0.2337,2.2500,2.1047,0.2189,0.0614,0.1451,0.0074,1.9499,2.2000,1.9655,0.9127,2.4537,2.3204,2.5000,0.1300,0.4984,0.0401,0.0197,0.1339,0.0671,2.3451,0.0689,0.0806,0.3137,2.4231,0.0024,0.0051,2.5,0.0012,0.0007,2.5000,0.0004,0.0595,0.1006,0.2553,1.4921,0.0007,0.0029,1.8558
3,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2020,0.0072,0.1653,2.5000,2.3125,0.0852,0.0587,0.0488,0.0083,2.2404,2.4242,2.2798,0.3017,2.5000,2.4933,2.5000,0.1185,0.5004,0.0385,0.0651,0.1390,0.0631,2.3385,0.0830,0.2679,0.1665,2.5000,0.1419,0.0013,2.5,0.0072,0.0216,0.6250,0.0413,0.0023,0.0006,0.1643,0.9946,0.0024,0.0026,1.8706
4,1705301000,Aborlan,Aborlan,Palawan,"aborlan, palawan",118.548417,9.437101,"9.4371009, 118.5484168",MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2021,0.0072,0.2606,1.2542,2.2948,0.0215,0.1064,0.0145,0.0061,2.1365,2.2619,2.3654,1.6250,2.4737,2.3721,1.2537,0.0835,0.0531,0.0060,0.1052,0.1231,0.0436,1.8169,0.0816,0.0909,0.0220,2.4545,0.0045,0.0067,2.5,0.0283,0.1089,1.8750,0.0690,0.0023,0.1846,0.1683,1.2502,0.1047,0.0030,1.2478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,806023000,Zumarraga,Zumarraga,Samar,"zumarraga, samar",124.841531,11.639055,"11.6390554, 124.8415308",MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,Zumarraga,2018,0.0123,0.0004,2.5000,1.4063,0.0021,0.0930,0.3131,0.7237,1.8189,2.3684,0.8120,1.0185,2.0000,1.5545,2.5000,0.1107,0.8730,0.0907,0.0073,0.1410,0.0909,1.4186,0.0864,0.3770,0.1720,1.2500,0.0010,0.0038,2.5,0.3350,0.0095,1.9984,0.1837,0.3131,0.0008,0.0569,1.5864,0.3079,0.0044,0.0000
9800,806023000,Zumarraga,Zumarraga,Samar,"zumarraga, samar",124.841531,11.639055

In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9804 entries, 0 to 9803
Data columns (total 52 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   psgc                                                 9804 non-null   int64  
 1   name                                                 9804 non-null   object 
 2   city_munic                                           9804 non-null   object 
 3   province                                             9804 non-null   object 
 4   clean_idx                                            9804 non-null   object 
 5   longitude                                            9804 non-null   float64
 6   latitude                                             9804 non-null   float64
 7   coords                                               9804 non-null   object 
 8   geometry                                             9804 non-null  

Lets remove some of the columns that are of no use.

In [26]:
col_list = ['name', 'psgc', 'city_munic', 'province', 'clean_idx', 'coords']
df2.drop(columns = col_list, inplace = True)
df2.head(8)

,longitude,latitude,geometry,FID,city_municipality,year,accommodation_capacity,active_establishments_in_the_locality,annual_disaster_drill,availability_of_basic_utilities,budget_for_drrmp,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,cost_of_doing_business,cost_of_living,disaster_risk_reduction_plan,distance_to_ports,early_warning_system,education,emergency_infrastructure,employed_population,employment_generation,financial_deepening,financial_technology_capacity,getting_business_permits,health,information_technology_capacity,lgu_investment,land_use_plan,local_economy_growth,local_economy_size,local_risk_assessments,peace_and_order,presence_of_business_and_professional_organizations,presence_of_investment_promotion_unit,productivity,recognition_of_performance,road_network,safety_compliant_business,sanitary_system,social_protection,transportation_vehicles,utilities
0,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2017,0.0072,0.2120,2.5000,2.5000,0.0017,0.1098,0.2879,1.4320,2.2597,2.3718,2.2277,1.5203,2.5000,2.1655,2.5000,0.0997,0.8750,0.0457,0.0218,0.1894,0.0463,0.7434,0.0623,0.2976,0.4592,1.8750,0.0012,0.0039,2.5,0.4161,0.0218,2.5000,0.0145,0.2879,0.1006,0.1943,1.5706,0.1104,0.0204,0.9949
1,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2018,0.0072,0.2337,2.5000,2.5000,0.0017,0.1098,0.2879,1.4320,2.2597,2.3718,2.1771,1.9907,2.5000,2.1655,2.5000,0.0997,0.8750,0.0457,0.0197,0.0423,0.0463,0.7434,0.0623,0.2976,0.4592,1.8750,0.0224,0.0072,2.5,0.4161,0.0218,2.5000,0.0047,0.2879,0.1006,0.1253,1.5706,0.1104,0.0204,0.0000
2,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2019,0.0072,0.2337,2.2500,2.1047,0.2189,0.0614,0.1451,0.0074,1.9499,2.2000,1.9655,0.9127,2.4537,2.3204,2.5000,0.1300,0.4984,0.0401,0.0197,0.1339,0.0671,2.3451,0.0689,0.0806,0.3137,2.4231,0.0024,0.0051,2.5,0.0012,0.0007,2.5000,0.0004,0.0595,0.1006,0.2553,1.4921,0.0007,0.0029,1.8558
3,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2020,0.0072,0.1653,2.5000,2.3125,0.0852,0.0587,0.0488,0.0083,2.2404,2.4242,2.2798,0.3017,2.5000,2.4933,2.5000,0.1185,0.5004,0.0385,0.0651,0.1390,0.0631,2.3385,0.0830,0.2679,0.1665,2.5000,0.1419,0.0013,2.5,0.0072,0.0216,0.6250,0.0413,0.0023,0.0006,0.1643,0.9946,0.0024,0.0026,1.8706
4,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2021,0.0072,0.2606,1.2542,2.2948,0.0215,0.1064,0.0145,0.0061,2.1365,2.2619,2.3654,1.6250,2.4737,2.3721,1.2537,0.0835,0.0531,0.0060,0.1052,0.1231,0.0436,1.8169,0.0816,0.0909,0.0220,2.4545,0.0045,0.0067,2.5,0.0283,0.1089,1.8750,0.0690,0.0023,0.1846,0.1683,1.2502,0.1047,0.0030,1.2478
5,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,Aborlan,2022,0.0072,0.0662,1.0020,0.0945,0.0215,0.0019,0.0022,0.0028,2.0000,1.8214,1.9004,1.9279,1.9998,1.8963,1.0087,0.0749,0.0292,0.0061,0.0624,0.0793,0.0172,1.5000,0.0978,0.0176,0.0012,1.9848,0.0001,0.0105,2.0,0.0644,0.0117,1.6667,0.0341,0.0023,0.0001,0.1683,1.0002,0.0014,0.0032,1.5007
6,120.726826,13.443721,POLYGON ((120.60896301200034 13.35233020800012...,1,Abra De Ilog,2017,0.0072,0.0479,2.5000,2.5000,0.0022,0.0748,0.1469,0.0483,1.9519,1.8214,2.2561,1.9257,2.5000,1.9287,2.5000,0.0518,0.8837,0.1280,0.0042,0.1263,0.0172,2.2085,0.0322,0.1915,0.3825,2.5000,0.0109,0.0089,2.5,0.4144,0.0860,1.6667,0.0278,0.1469,0.0002,0.0369,1.0235,0.0947,0.0476,1.0076
7,120.726826,13.443721,POLYGON ((120.60896301200034 13.35233020800012...,1,Abra De Ilog,2018,0.0072,0.0477,2.5000,2.5000,0.0022,0.0748,0.1469,0.0483,1.9519,1.8214,2.2200,2.3611,2.5000,1.9287,2.5000,0.0518,0.8837,0.1280,0.0008,0.0184,0.0172,2.2085,0.0322,0.1915,0.3825,2.5000,0.0102,0.0027,2.5,0.4144,0.0235,1.6667,0.0579,0.1469,0.0002,0.0282,1.0235,0.0947,0.0476,0.0000


In [27]:
df2.shape

(9804, 46)

In [28]:
df2.columns.to_list()

['longitude',
 'latitude',
 'geometry',
 'FID',
 'city_municipality',
 'year',
 'accommodation_capacity',
 'active_establishments_in_the_locality',
 'annual_disaster_drill',
 'availability_of_basic_utilities',
 'budget_for_drrmp',
 'capacity_of_health_services',
 'capacity_of_school_services',
 'capacity_to_generate_local_resource',
 'compliance_to_arta_citizens_charter',
 'compliance_to_national_directives',
 'cost_of_doing_business',
 'cost_of_living',
 'disaster_risk_reduction_plan',
 'distance_to_ports',
 'early_warning_system',
 'education',
 'emergency_infrastructure',
 'employed_population',
 'employment_generation',
 'financial_deepening',
 'financial_technology_capacity',
 'getting_business_permits',
 'health',
 'information_technology_capacity',
 'lgu_investment',
 'land_use_plan',
 'local_economy_growth',
 'local_economy_size',
 'local_risk_assessments',
 'peace_and_order',
 'presence_of_business_and_professional_organizations',
 'presence_of_investment_promotion_unit',
 'pr

In [29]:
cols = [
    'city_municipality', 'longitude', 'latitude', 'geometry', 'FID', 'year',
	'active_establishments_in_the_locality',
 	'cost_of_doing_business',
    'cost_of_living', 'employment_generation', 'financial_deepening',
	'local_economy_growth', 'local_economy_size',
	'presence_of_business_and_professional_organizations', 'productivity',
	'safety_compliant_business',
	'capacity_of_health_services', 'capacity_of_school_services',
	'capacity_to_generate_local_resource',
	'compliance_to_arta_citizens_charter',
	'compliance_to_national_directives', 'getting_business_permits',
	'peace_and_order', 'presence_of_investment_promotion_unit',
	'recognition_of_performance', 'social_protection',
	'annual_disaster_drill', 'budget_for_drrmp',
	'disaster_risk_reduction_plan', 'early_warning_system',
	'emergency_infrastructure', 'employed_population', 'land_use_plan',
	'local_risk_assessments', 'sanitary_system', 'utilities',
    'accommodation_capacity', 'availability_of_basic_utilities',
	'distance_to_ports', 'education', 'financial_technology_capacity',
	'health', 'information_technology_capacity', 'lgu_investment',
	'road_network', 'transportation_vehicles' ]

In [30]:
df2 = df2[cols]
df2

,city_municipality,longitude,latitude,geometry,FID,year,active_establishments_in_the_locality,cost_of_doing_business,cost_of_living,employment_generation,financial_deepening,local_economy_growth,local_economy_size,presence_of_business_and_professional_organizations,productivity,safety_compliant_business,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,getting_business_permits,peace_and_order,presence_of_investment_promotion_unit,recognition_of_performance,social_protection,annual_disaster_drill,budget_for_drrmp,disaster_risk_reduction_plan,early_warning_system,emergency_infrastructure,employed_population,land_use_plan,local_risk_assessments,sanitary_system,utilities,accommodation_capacity,availability_of_basic_utilities,distance_to_ports,education,financial_technology_capacity,health,information_technology_capacity,lgu_investment,road_network,transportation_vehicles
0,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2017,0.2120,2.2277,1.5203,0.0218,0.1894,0.0012,0.0039,0.0218,0.0145,0.1943,0.1098,0.2879,1.4320,2.2597,2.3718,0.7434,0.4161,2.5000,0.2879,0.1104,2.5000,0.0017,2.5000,2.5000,0.8750,0.0457,1.8750,2.5,1.5706,0.9949,0.0072,2.5000,2.1655,0.0997,0.0463,0.0623,0.2976,0.4592,0.1006,0.0204
1,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2018,0.2337,2.1771,1.9907,0.0197,0.0423,0.0224,0.0072,0.0218,0.0047,0.1253,0.1098,0.2879,1.4320,2.2597,2.3718,0.7434,0.4161,2.5000,0.2879,0.1104,2.5000,0.0017,2.5000,2.5000,0.8750,0.0457,1.8750,2.5,1.5706,0.0000,0.0072,2.5000,2.1655,0.0997,0.0463,0.0623,0.2976,0.4592,0.1006,0.0204
2,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2019,0.2337,1.9655,0.9127,0.0197,0.1339,0.0024,0.0051,0.0007,0.0004,0.2553,0.0614,0.1451,0.0074,1.9499,2.2000,2.3451,0.0012,2.5000,0.0595,0.0007,2.2500,0.2189,2.4537,2.5000,0.4984,0.0401,2.4231,2.5,1.4921,1.8558,0.0072,2.1047,2.3204,0.1300,0.0671,0.0689,0.0806,0.3137,0.1006,0.0029
3,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2020,0.1653,2.2798,0.3017,0.0651,0.1390,0.1419,0.0013,0.0216,0.0413,0.1643,0.0587,0.0488,0.0083,2.2404,2.4242,2.3385,0.0072,0.6250,0.0023,0.0024,2.5000,0.0852,2.5000,2.5000,0.5004,0.0385,2.5000,2.5,0.9946,1.8706,0.0072,2.3125,2.4933,0.1185,0.0631,0.0830,0.2679,0.1665,0.0006,0.0026
4,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2021,0.2606,2.3654,1.6250,0.1052,0.1231,0.0045,0.0067,0.1089,0.0690,0.1683,0.1064,0.0145,0.0061,2.1365,2.2619,1.8169,0.0283,1.8750,0.0023,0.1047,1.2542,0.0215,2.4737,1.2537,0.0531,0.0060,2.4545,2.5,1.2502,1.2478,0.0072,2.2948,2.3721,0.0835,0.0436,0.0816,0.0909,0.0220,0.1846,0.0030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,Zumarraga,124.841531,11.639055,MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,2018,0.0004,0.8120,1.0185,0.0073,0.1410,0.0010,0.0038,0.0095,0.1837,0.0569,0.0930,0.3131,0.7237,1.8189,2.3684,1.4186,0.3350,1.9984,0.3131,0.3079,2.5000,0.0021,2.0000,2.5000,0.8730,0.0907,1.2500,2.5,1.5864,0.0000,0.0123,1.4063,1.5545,0.1107,0.0909,0.0864,0.3770,0.1720,0.0008,0.0044
9800,Zumarraga,124.841531,11.639055,MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,2019,0.0093,1.4285,1.0000,0.0060,0.1410,0.0010,0.0038,0.0095,0.0033,0.0569,0.0324,0.2339,0.7070,2.3612,2.4994,1.5524,0.1843,1.9984,0.3131,0.7160,2.4107,0.0632,2.4537,2.5000,0.4590,0.0073,2.0000,2.5,1.0144,1.8799,0.0104,1.4719,2.4148,0.0473,0.0909,0.0277,0.2097,0.0435,0.0202,0.0014
9801,Zumarraga,124.841531,11.639055,MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,2020,1.0019,1.5196,1.6803,0.0385,0.0174,0.0090,0.0038,0.0027,0.0298,0.0569,0.1629,0.1926,0.3451,2.3957,2.5000,1.4931,0.5482,1.2500,0.2262,0.1281,2.5000,0.0636,2.50

In [31]:
df3 = df2.copy()

In [32]:
df3

,city_municipality,longitude,latitude,geometry,FID,year,active_establishments_in_the_locality,cost_of_doing_business,cost_of_living,employment_generation,financial_deepening,local_economy_growth,local_economy_size,presence_of_business_and_professional_organizations,productivity,safety_compliant_business,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,getting_business_permits,peace_and_order,presence_of_investment_promotion_unit,recognition_of_performance,social_protection,annual_disaster_drill,budget_for_drrmp,disaster_risk_reduction_plan,early_warning_system,emergency_infrastructure,employed_population,land_use_plan,local_risk_assessments,sanitary_system,utilities,accommodation_capacity,availability_of_basic_utilities,distance_to_ports,education,financial_technology_capacity,health,information_technology_capacity,lgu_investment,road_network,transportation_vehicles
0,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2017,0.2120,2.2277,1.5203,0.0218,0.1894,0.0012,0.0039,0.0218,0.0145,0.1943,0.1098,0.2879,1.4320,2.2597,2.3718,0.7434,0.4161,2.5000,0.2879,0.1104,2.5000,0.0017,2.5000,2.5000,0.8750,0.0457,1.8750,2.5,1.5706,0.9949,0.0072,2.5000,2.1655,0.0997,0.0463,0.0623,0.2976,0.4592,0.1006,0.0204
1,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2018,0.2337,2.1771,1.9907,0.0197,0.0423,0.0224,0.0072,0.0218,0.0047,0.1253,0.1098,0.2879,1.4320,2.2597,2.3718,0.7434,0.4161,2.5000,0.2879,0.1104,2.5000,0.0017,2.5000,2.5000,0.8750,0.0457,1.8750,2.5,1.5706,0.0000,0.0072,2.5000,2.1655,0.0997,0.0463,0.0623,0.2976,0.4592,0.1006,0.0204
2,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2019,0.2337,1.9655,0.9127,0.0197,0.1339,0.0024,0.0051,0.0007,0.0004,0.2553,0.0614,0.1451,0.0074,1.9499,2.2000,2.3451,0.0012,2.5000,0.0595,0.0007,2.2500,0.2189,2.4537,2.5000,0.4984,0.0401,2.4231,2.5,1.4921,1.8558,0.0072,2.1047,2.3204,0.1300,0.0671,0.0689,0.0806,0.3137,0.1006,0.0029
3,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2020,0.1653,2.2798,0.3017,0.0651,0.1390,0.1419,0.0013,0.0216,0.0413,0.1643,0.0587,0.0488,0.0083,2.2404,2.4242,2.3385,0.0072,0.6250,0.0023,0.0024,2.5000,0.0852,2.5000,2.5000,0.5004,0.0385,2.5000,2.5,0.9946,1.8706,0.0072,2.3125,2.4933,0.1185,0.0631,0.0830,0.2679,0.1665,0.0006,0.0026
4,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2021,0.2606,2.3654,1.6250,0.1052,0.1231,0.0045,0.0067,0.1089,0.0690,0.1683,0.1064,0.0145,0.0061,2.1365,2.2619,1.8169,0.0283,1.8750,0.0023,0.1047,1.2542,0.0215,2.4737,1.2537,0.0531,0.0060,2.4545,2.5,1.2502,1.2478,0.0072,2.2948,2.3721,0.0835,0.0436,0.0816,0.0909,0.0220,0.1846,0.0030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,Zumarraga,124.841531,11.639055,MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,2018,0.0004,0.8120,1.0185,0.0073,0.1410,0.0010,0.0038,0.0095,0.1837,0.0569,0.0930,0.3131,0.7237,1.8189,2.3684,1.4186,0.3350,1.9984,0.3131,0.3079,2.5000,0.0021,2.0000,2.5000,0.8730,0.0907,1.2500,2.5,1.5864,0.0000,0.0123,1.4063,1.5545,0.1107,0.0909,0.0864,0.3770,0.1720,0.0008,0.0044
9800,Zumarraga,124.841531,11.639055,MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,2019,0.0093,1.4285,1.0000,0.0060,0.1410,0.0010,0.0038,0.0095,0.0033,0.0569,0.0324,0.2339,0.7070,2.3612,2.4994,1.5524,0.1843,1.9984,0.3131,0.7160,2.4107,0.0632,2.4537,2.5000,0.4590,0.0073,2.0000,2.5,1.0144,1.8799,0.0104,1.4719,2.4148,0.0473,0.0909,0.0277,0.2097,0.0435,0.0202,0.0014
9801,Zumarraga,124.841531,11.639055,MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,2020,1.0019,1.5196,1.6803,0.0385,0.0174,0.0090,0.0038,0.0027,0.0298,0.0569,0.1629,0.1926,0.3451,2.3957,2.5000,1.4931,0.5482,1.2500,0.2262,0.1281,2.5000,0.0636,2.50

In [ ]:
print(df3.columns[[8, 17, 18, 25, 26, 33, 34, 37, 39, 42]])

In [ ]:
print(df3.columns[[12, 13, 14, 15, 16, 28, 36, 38, 40, 44]])

In [ ]:
print(df3.columns[[9, 11, 19, 21, 23, 24, 32, 35, 43, 46]])

In [ ]:
print(df3.columns[[7, 10, 20, 22, 27, 29, 30, 31, 41, 45]])

# PCA - MEAN

In [33]:
columns = df3.columns.to_list()

In [34]:
columns

['city_municipality',
 'longitude',
 'latitude',
 'geometry',
 'FID',
 'year',
 'active_establishments_in_the_locality',
 'cost_of_doing_business',
 'cost_of_living',
 'employment_generation',
 'financial_deepening',
 'local_economy_growth',
 'local_economy_size',
 'presence_of_business_and_professional_organizations',
 'productivity',
 'safety_compliant_business',
 'capacity_of_health_services',
 'capacity_of_school_services',
 'capacity_to_generate_local_resource',
 'compliance_to_arta_citizens_charter',
 'compliance_to_national_directives',
 'getting_business_permits',
 'peace_and_order',
 'presence_of_investment_promotion_unit',
 'recognition_of_performance',
 'social_protection',
 'annual_disaster_drill',
 'budget_for_drrmp',
 'disaster_risk_reduction_plan',
 'early_warning_system',
 'emergency_infrastructure',
 'employed_population',
 'land_use_plan',
 'local_risk_assessments',
 'sanitary_system',
 'utilities',
 'accommodation_capacity',
 'availability_of_basic_utilities',
 'dist

In [34]:
columns_to_use = ['active_establishments_in_the_locality', 'cost_of_doing_business',
       'cost_of_living', 'employment_generation', 'financial_deepening',
       'local_economy_growth', 'local_economy_size',
       'presence_of_business_and_professional_organizations', 'productivity',
       'safety_compliant_business','capacity_of_health_services', 'capacity_of_school_services',
       'capacity_to_generate_local_resource',
       'compliance_to_arta_citizens_charter',
       'compliance_to_national_directives', 'getting_business_permits',
       'peace_and_order', 'presence_of_investment_promotion_unit',
       'recognition_of_performance', 'social_protection', 'accommodation_capacity', 'availability_of_basic_utilities',
       'distance_to_ports', 'education', 'financial_technology_capacity',
       'health', 'information_technology_capacity', 'lgu_investment',
       'road_network', 'transportation_vehicles', 'annual_disaster_drill', 'budget_for_drrmp',
       'disaster_risk_reduction_plan', 'early_warning_system',
       'emergency_infrastructure', 'employed_population', 'land_use_plan',
       'local_risk_assessments', 'sanitary_system', 'utilities']

In [35]:
data = df3[columns_to_use]
columns_names = columns

In [35]:
economic_dynamism = ['active_establishments_in_the_locality', 'cost_of_doing_business',
       'cost_of_living', 'employment_generation', 'financial_deepening',
       'local_economy_growth', 'local_economy_size',
       'presence_of_business_and_professional_organizations', 'productivity',
       'safety_compliant_business']
government_efficiency = ['capacity_of_health_services', 'capacity_of_school_services',
       'capacity_to_generate_local_resource',
       'compliance_to_arta_citizens_charter',
       'compliance_to_national_directives', 'getting_business_permits',
       'peace_and_order', 'presence_of_investment_promotion_unit',
       'recognition_of_performance', 'social_protection']
infrastructure = ['accommodation_capacity', 'availability_of_basic_utilities',
       'distance_to_ports', 'education', 'financial_technology_capacity',
       'health', 'information_technology_capacity', 'lgu_investment',
       'road_network', 'transportation_vehicles']
resiliency = ['annual_disaster_drill', 'budget_for_drrmp',
       'disaster_risk_reduction_plan', 'early_warning_system',
       'emergency_infrastructure', 'employed_population', 'land_use_plan',
       'local_risk_assessments', 'sanitary_system', 'utilities']

In [36]:
pca_1 = df3[economic_dynamism]
pca_2 = df3[government_efficiency]
pca_3 = df3[infrastructure]
pca_4 = df3[resiliency]

In [37]:
for i, df4 in enumerate([pca_1, pca_2, pca_3, pca_4]):
    data = df4
    pca = PCA(n_components=1)
    pca.fit(data)
    reduced_data = pca.transform(data)
    df3["PCA{}".format(i+1)] = reduced_data[:, 0]
    print("PCA results for dataframe {}:".format(i+1))
    print("Explained variance: ", pca.explained_variance_ratio_)

PCA results for dataframe 1:
Explained variance:  [0.30773347]
PCA results for dataframe 2:
Explained variance:  [0.30046224]
PCA results for dataframe 3:
Explained variance:  [0.65726871]
PCA results for dataframe 4:
Explained variance:  [0.45538798]


In [38]:
df3

,city_municipality,longitude,latitude,geometry,FID,year,active_establishments_in_the_locality,cost_of_doing_business,cost_of_living,employment_generation,financial_deepening,local_economy_growth,local_economy_size,presence_of_business_and_professional_organizations,productivity,safety_compliant_business,capacity_of_health_services,capacity_of_school_services,capacity_to_generate_local_resource,compliance_to_arta_citizens_charter,compliance_to_national_directives,getting_business_permits,peace_and_order,presence_of_investment_promotion_unit,recognition_of_performance,social_protection,annual_disaster_drill,budget_for_drrmp,disaster_risk_reduction_plan,early_warning_system,emergency_infrastructure,employed_population,land_use_plan,local_risk_assessments,sanitary_system,utilities,accommodation_capacity,availability_of_basic_utilities,distance_to_ports,education,financial_technology_capacity,health,information_technology_capacity,lgu_investment,road_network,transportation_vehicles,PCA1,PCA2,PCA3,PCA4
0,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2017,0.2120,2.2277,1.5203,0.0218,0.1894,0.0012,0.0039,0.0218,0.0145,0.1943,0.1098,0.2879,1.4320,2.2597,2.3718,0.7434,0.4161,2.5000,0.2879,0.1104,2.5000,0.0017,2.5000,2.5000,0.8750,0.0457,1.8750,2.5,1.5706,0.9949,0.0072,2.5000,2.1655,0.0997,0.0463,0.0623,0.2976,0.4592,0.1006,0.0204,-0.004461,-0.028991,-0.798628,-0.872007
1,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2018,0.2337,2.1771,1.9907,0.0197,0.0423,0.0224,0.0072,0.0218,0.0047,0.1253,0.1098,0.2879,1.4320,2.2597,2.3718,0.7434,0.4161,2.5000,0.2879,0.1104,2.5000,0.0017,2.5000,2.5000,0.8750,0.0457,1.8750,2.5,1.5706,0.0000,0.0072,2.5000,2.1655,0.0997,0.0463,0.0623,0.2976,0.4592,0.1006,0.0204,-0.479777,-0.028991,-0.798628,-0.902937
2,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2019,0.2337,1.9655,0.9127,0.0197,0.1339,0.0024,0.0051,0.0007,0.0004,0.2553,0.0614,0.1451,0.0074,1.9499,2.2000,2.3451,0.0012,2.5000,0.0595,0.0007,2.2500,0.2189,2.4537,2.5000,0.4984,0.0401,2.4231,2.5,1.4921,1.8558,0.0072,2.1047,2.3204,0.1300,0.0671,0.0689,0.0806,0.3137,0.1006,0.0029,0.555866,-0.528242,-0.410436,-0.735777
3,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2020,0.1653,2.2798,0.3017,0.0651,0.1390,0.1419,0.0013,0.0216,0.0413,0.1643,0.0587,0.0488,0.0083,2.2404,2.4242,2.3385,0.0072,0.6250,0.0023,0.0024,2.5000,0.0852,2.5000,2.5000,0.5004,0.0385,2.5000,2.5,0.9946,1.8706,0.0072,2.3125,2.4933,0.1185,0.0631,0.0830,0.2679,0.1665,0.0006,0.0026,1.211439,0.788142,-0.652061,-0.900264
4,Aborlan,118.548417,9.437101,MULTIPOLYGON (((118.57997900000032 9.372150000...,0,2021,0.2606,2.3654,1.6250,0.1052,0.1231,0.0045,0.0067,0.1089,0.0690,0.1683,0.1064,0.0145,0.0061,2.1365,2.2619,1.8169,0.0283,1.8750,0.0023,0.1047,1.2542,0.0215,2.4737,1.2537,0.0531,0.0060,2.4545,2.5,1.2502,1.2478,0.0072,2.2948,2.3721,0.0835,0.0436,0.0816,0.0909,0.0220,0.1846,0.0030,-0.087736,0.155330,-0.571234,0.799489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9799,Zumarraga,124.841531,11.639055,MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,2018,0.0004,0.8120,1.0185,0.0073,0.1410,0.0010,0.0038,0.0095,0.1837,0.0569,0.0930,0.3131,0.7237,1.8189,2.3684,1.4186,0.3350,1.9984,0.3131,0.3079,2.5000,0.0021,2.0000,2.5000,0.8730,0.0907,1.2500,2.5,1.5864,0.0000,0.0123,1.4063,1.5545,0.1107,0.0909,0.0864,0.3770,0.1720,0.0008,0.0044,0.283300,0.118666,0.393224,-0.682634
9800,Zumarraga,124.841531,11.639055,MULTIPOLYGON (((124.87373400000024 11.62500000...,1633,2019,0.0093,1.4285,1.0000,0.0060,0.1410,0.0010,0.0038,0.0095,0.0033,0.0569,0.0324,0.2339,0.7070,2.3612,2.4994,1.5524,0.1843,1.9984,0.3131,0.7160,2.4107,0.0632,2.4537,2.5000,0.4590,0.0073,2.0000,2.5,1.0144,1.8799,0.0104,1.4719,2.4148,0.0473,0.0909,0.0277,0.2097,0.0435,

# PCA - SUM

In [ ]:
indexed_data_sum = []
for group in grouped_data:
    indexed_data_sum.append(np.sum(group, axis=1))

In [ ]:
final_data = np.hstack((np.array(indexed_data_sum).T, reduced_data))

In [ ]:
df7 = pd.DataFrame(final_data, columns=column_names)
df7

In [ ]:
df8 = pd.concat([df5, df7],axis=1, join='inner')
df8

In [ ]:
df4 = pd.DataFrame(final_data, columns=columns)

In [ ]:
group1_economic_dynamism = df3.iloc[8, 17, 18, 25, 26, 33, 34, 37, 39, 42]
group2_government_efficiency = df3.iloc[12, 13, 14, 15, 16, 28, 36, 38, 40, 44]
group3_infrastructure = df3.iloc[7, 10, 20, 22, 27, 29, 30, 31, 41, 45]
group4_resiliency = df3.iloc[9, 11, 19, 21, 23, 24, 32, 35, 43, 46]

In [ ]:
group1_economic_dynamism = df3.loc[
    (df3.index.isin([8, 17, 18, 25, 26, 33, 34, 37, 39, 42]))]
group2_government_efficiency = df3.loc[
    (df3.index.isin([12, 13, 14, 15, 16, 28, 36, 38, 40, 44]))]
group3_infrastructure = df3.loc[
    (df3.index.isin([7, 10, 20, 22, 27, 29, 30, 31, 41, 45]))]
group4_resiliency = df3.loc[
    (df3.index.isin([9, 11, 19, 21, 23, 24, 32, 35, 43, 46]))]

In [ ]:
grouped_data = []
grouped_data.append(df3.loc[:, group1_economic_dynamism])
grouped_data.append(df3.loc[:, group2_government_efficiency])
grouped_data.append(df3.loc[:, group3_infrastructure])
grouped_data.append(df3.loc[:, group4_resiliency])

In [ ]:
grouped_data = []
grouped_data.append(df3.loc[:, economic_dynamism])
grouped_data.append(df3.loc[:, government_efficiency])
grouped_data.append(df3.loc[:, infrastructure])
grouped_data.append(df3.loc[:, resiliency])

In [ ]:
indexed_data = []
for group in grouped_data:
    indexed_data.append(np.mean(group, axis=1))

In [ ]:
final_data = np.hstack((np.array(indexed_data).T, reduced_data))
final_data

In [ ]:
final_data = np.hstack((np.array(indexed_data).T, reduced_data))
final_data = pd.concat([economic_dynamism, government_efficiency, infrastructure, resiliency, final_data], axis=1)
final_data

In [ ]:
columns = [
    'name',
    'longitude',	'latitude',	'geometry', 'FID', 'city_municipality', 'year', 'economic_dynamism', 'government_efficiency', 'infrastructure', 'resiliency'] + ['PCA{}'.format(i+1) for i in range(4)]
df4 = pd.DataFrame(final_data, columns=columns)
df4

In [ ]:
columns = [
    'economic_dynamism',
    'government_efficiency',
    'infrastructure',
    'resiliency'] + [
        'PCA{}'.format(i+1) for i in range(4)] + [
            col for i, col in enumerate(columns_names)
            if i not in columns_to_use
            +df3.iloc[group1_economic_dynamism]
            +df3.iloc[group2_government_efficiency]
            +df3.iloc[group3_infrastructure]
            +df3.iloc[group4_resiliency]]
df4 = pd.DataFrame(final_data, columns=columns)
df4

In [ ]:
group1_economic_dynamism

In [ ]:
sns.lineplot(x='year', y='employed_population', data=df2)
plt.title("Employed population by year")
plt.xlabel("Year")
plt.ylabel("Employed population Indicator")
plt.show;

In [ ]:
sns.lineplot(x='name', y='employed_population', data=df2)
plt.title("Employed population by year")
plt.xlabel("Year")
plt.ylabel("Employed population Indicator")
plt.show;

In [ ]:
def map_ph(df2, year, pforce):
    cond = (df2['year'] == year) & (df2['police_force'] == pforce)

    lat = df2[cond]['latitude'].tolist()
    lon = df2[cond]['longitude'].tolist()
    nam = df2[cond]['police_force'].tolist()
    sev = df2[cond]['accident_severity'].tolist()
    cas = df2[cond]['number_of_casualties'].tolist()
    veh = df2[cond]['number_of_vehicles'].tolist()
    dat = df2[cond]['date'].tolist()

    def color_producer(status):
        if 'Slight' in status:
            return 'green'
        elif 'Serious' in status:
            return 'blue'
        else:
            return 'orange'

    html = '''<h4>Collision Information</h4>
    <b>%s</b> <br /><br />
    <b>Severity: </b> %s <br />
    <b>Casualties: </b> %s <br />
    <b>Vehicles: </b> %s <br />
    <b>Date: </b> %s
    '''
    map = flm.Map(location=[lat[1], lon[1]], zoom_start=12, scrollWheelZoom=False)

    fg = flm.FeatureGroup(name='My V Map')

    for lt, ln, nm, st, ca, ve, da in zip((lat), (lon), (nam), (sev), (cas), (veh), (dat)):
        iframe = flm.IFrame(html = html % ((nm), (st), (ca), (ve), (da)), height = 165)
        popup = flm.Popup(iframe, min_width=200, max_width=500)
        fg.add_child(flm.CircleMarker(location = [lt, ln], popup = (popup), fill_color=color_producer(st), color='None', radius=10, fill_opacity = 0.7))
        map.add_child(fg)

    map.save('C:/Users/kmedr/My Drive/Omdena_Predicting_RTC_main/reports/map.html')
    return map